In [23]:
import pandas as pd
import numpy as np

In [24]:
# path_to_mpa_output="/shared/projects/mudis4ls_is4_benchmark/test_results/compare_outputs/220/mpa_220.tsv"
path_to_sylph_output="/shared/projects/mudis4ls_is4_benchmark/test_results/compare_outputs/220/sylph_220.tsv"
# path_to_motus_output="/shared/projects/mudis4ls_is4_benchmark/test_results/compare_outputs/220/motus_220.tsv"
# path_to_bracken_output="/shared/projects/mudis4ls_is4_benchmark/test_results/kraken/gtdb/test_Species.bracken"
path_to_meteor_output="/shared/projects/mudis4ls_is4_benchmark/test_results/meteor/profile/ERS12377136/ERS12377136_msp.tsv"


In [25]:
path_to_gtdb_msp_meteor="/shared/projects/mudis4ls_is4_benchmark/BDs/hs_10_4_gut/database/hs_10_4_gut_1990MSPs_gtdb_r220.feather"


In [26]:
gtdb_msp_meteor=pd.read_feather(path_to_gtdb_msp_meteor)

In [27]:
# Charger les fichiers en DataFrames
# df_metaphlan = pd.read_csv(path_to_mpa_output, sep="\t" )
df_sylph = pd.read_csv(path_to_sylph_output, sep="\t")
# df_motus= pd.read_csv(path_to_motus_output, sep="\t")

In [28]:
# Renommer la colonne relative_abundance pour chaque outil

df_sylph.rename(columns={"relative_abundance": "abundance_sylph"}, inplace=True)


In [29]:
df_sylph.rename(columns={"clade_name": "gtdb_r220_classification"}, inplace=True)

In [30]:
# Définition des niveaux taxonomiques
tax_levels = ["superkingdom", "phylum", "class", "order", "family", "genus", "species"]

# Expression régulière pour extraire chaque niveau taxonomique
regex = r"d__(?P<superkingdom>[^;]+)(?:;p__(?P<phylum>[^;]+))?(?:;c__(?P<class>[^;]+))?(?:;o__(?P<order>[^;]+))?(?:;f__(?P<family>[^;]+))?(?:;g__(?P<genus>[^;]+))?(?:;s__(?P<species>[^;]+))?"

# Extraction des valeurs avec str.extract()
df_sylph[tax_levels] = df_sylph["gtdb_r220_classification"].str.extract(regex)

# Remplacer les valeurs manquantes par NaN (optionnel, c'est déjà le comportement par défaut)
df_sylph.fillna(np.nan, inplace=True)

In [31]:
#mapper les msp de meteor avec la taxonomie de gtdb 220
df_meteor = pd.read_csv(path_to_meteor_output, sep="\t")

In [32]:
df_meteor.rename(columns={"value": "abundance_meteor"}, inplace=True)

In [33]:
df_meteor = df_meteor[df_meteor["abundance_meteor"] != 0.0]


In [34]:
gtdb_msp_meteor
#l'annotation gtdb de toutes les msp possibles par meteor

,msp_name,gtdb_r220_classification,superkingdom,phylum,class,order,family,genus,species,msp
0,msp_0001,d__Eukaryota;p__Heterokonta;c__Blastocystae;o_...,Eukaryota,Heterokonta,Blastocystae,Blastocystida,Blastocystidae,Blastocystis,Blastocystis sp. subtype 3,msp_0001
1,msp_0002,d__Eukaryota;p__Heterokonta;c__Blastocystae;o_...,Eukaryota,Heterokonta,Blastocystae,Blastocystida,Blastocystidae,Blastocystis,Blastocystis sp. subtype 1,msp_0002
2,msp_0003,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides cellulosilyticus,msp_0003
3,msp_0004,d__Eukaryota;p__Heterokonta;c__Blastocystae;o_...,Eukaryota,Heterokonta,Blastocystae,Blastocystida,Blastocystidae,Blastocystis,Blastocystis sp. subtype 2,msp_0004
4,msp_0005,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Escherichia,Escherichia coli,msp_0005
...,...,...,...,...,...,...,...,...,...,...
1985,msp_2196,d__Bacteria;p__Bacillota_A;c__Clostridia;o__Os...,Bacteria,Bacillota_A,Clostridia,Oscillospirales,Acutalibacteraceae,Eubacterium_R,Eubacterium_R sp014465955,msp_2196
1986,msp_2207,d__Bacteria;p__Bacillota_A;c__Clostridia;o__Os...,Bacteria,Bacillota_A,Clostridia,Oscillospirales,Butyricicoccaceae,Butyricicoccaceae SIG471,Butyricicoccaceae SIG471 sp905199135,msp_2207
1987,msp_2209,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales,Succinivibrionaceae,Succinatimonas,Succinatimonas sp905203285,msp_2209
1988,msp_2210,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides sp. bacterium,msp_2210


In [35]:
df_meteor=df_meteor.merge(gtdb_msp_meteor, on = "msp_name",how="left")

In [36]:
df_meteor
#les msp assignés par meteor avec leur taxonomie gtdb, pour l'échantillon test

,msp_name,abundance_meteor,gtdb_r220_classification,superkingdom,phylum,class,order,family,genus,species,msp
0,msp_0007,0.022924,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides ovatus,msp_0007
1,msp_0009,0.011958,d__Bacteria;p__Bacillota_A;c__Clostridia;o__La...,Bacteria,Bacillota_A,Clostridia,Lachnospirales,Lachnospiraceae,Enterocloster,Enterocloster bolteae,msp_0009
2,msp_0012,0.016823,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Tannerellaceae,Parabacteroides,Parabacteroides distasonis,msp_0012
3,msp_0017,0.037285,d__Bacteria;p__Bacillota_A;c__Clostridia;o__La...,Bacteria,Bacillota_A,Clostridia,Lachnospirales,Lachnospiraceae,Roseburia,Roseburia intestinalis,msp_0017
4,msp_0018,0.015694,d__Bacteria;p__Bacillota_A;c__Clostridia;o__La...,Bacteria,Bacillota_A,Clostridia,Lachnospirales,Lachnospiraceae,Eisenbergiella,Eisenbergiella porci,msp_0018
...,...,...,...,...,...,...,...,...,...,...,...
62,msp_0735,0.004740,d__Bacteria;p__Bacillota_A;c__Clostridia;o__Mo...,Bacteria,Bacillota_A,Clostridia,Monoglobales,Monoglobaceae,Monoglobus,Monoglobus pectinilyticus,msp_0735
63,msp_0740,0.006924,d__Bacteria;p__Bacillota_A;c__Clostridia;o__La...,Bacteria,Bacillota_A,Clostridia,Lachnospirales,Anaerotignaceae,Metalachnospira,Metalachnospira sp900548595,msp_0740
64,msp_0986,0.005266,d__Bacteria;p__Bacillota_A;c__Clostridia;o__La...,Bacteria,Bacillota_A,Clostridia,Lachnospirales,Lachnospiraceae,Coprococcus_A,Coprococcus_A catus_A,msp_0986
65,msp_1033,0.016690,d__Bacteria;p__Bacillota_A;c__Clostridia;o__TA...,Bacteria,Bacillota_A,Clostridia,Clostridia TANB77,Clostridia TANB77 CAG-508,Clostridia TANB77 CAG-508 CAG-273,Clostridia TANB77 CAG-508 CAG-273 sp000438355,msp_1033


In [37]:
meteor_sylph = df_meteor.merge(
    df_sylph, on = (
        "gtdb_r220_classification","superkingdom","phylum","class","order","family","genus","species"
    ),how="outer"
    )

In [38]:
meteor_sylph

,msp_name,abundance_meteor,gtdb_r220_classification,superkingdom,phylum,class,order,family,genus,species,msp,abundance_sylph,sequence_abundance,ANI (if strain-level),Coverage (if strain-level)
0,NaN,NaN,d__Bacteria,Bacteria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0000,100.0001,NaN,NaN
1,msp_0584,0.006384,d__Bacteria;p__Actinomycetota;c__Coriobacterii...,Bacteria,Actinomycetota,Coriobacteriia,Coriobacteriales,Eggerthellaceae,Anaerotardibacter,Anaerotardibacter sp000435675,msp_0584,NaN,NaN,NaN,NaN
2,msp_0573,0.006947,d__Bacteria;p__Actinomycetota;c__Coriobacterii...,Bacteria,Actinomycetota,Coriobacteriia,Coriobacteriales,Eggerthellaceae,Eggerthella,Eggerthella lenta,msp_0573,NaN,NaN,NaN,NaN
3,NaN,NaN,d__Bacteria;p__Bacillota_A,Bacteria,Bacillota_A,NaN,NaN,NaN,NaN,NaN,NaN,41.4963,36.1829,NaN,NaN
4,NaN,NaN,d__Bacteria;p__Bacillota_A;c__Clostridia,Bacteria,Bacillota_A,Clostridia,NaN,NaN,NaN,NaN,NaN,41.4963,36.1829,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,NaN,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,NaN,NaN,NaN,1.3124,0.9629,NaN,NaN
135,NaN,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Sutterella,NaN,NaN,1.3124,0.9629,NaN,NaN
136,msp_0280,0.030240,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Sutterella,Sutterella wadsworthensis_A,msp_0280,1.3124,0.9629,NaN,NaN
137,NaN,NaN,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Sutterella,Sutterella wadsworthensis_A,NaN,1.3124,0.9629,98.48,0.035


In [39]:
meteor_sylph.to_csv(
    "/shared/projects/mudis4ls_is4_benchmark/test_results/compare_outputs/sylph_meteor_bracken/meteor_sylph.tsv", sep="\t", index=False
    )

In [ ]:
df_sylph

,gtdb_r220_classification,abundance_sylph,sequence_abundance,ANI (if strain-level),Coverage (if strain-level)
0,d__Bacteria,100.0000,100.0001,NaN,NaN
1,d__Bacteria;p__Bacteroidota,50.7665,59.3626,NaN,NaN
2,d__Bacteria;p__Bacillota_A,41.4963,36.1829,NaN,NaN
3,d__Bacteria;p__Pseudomonadota,4.3007,2.3112,NaN,NaN
4,d__Bacteria;p__Bacillota_C,3.4365,2.1434,NaN,NaN
...,...,...,...,...,...
88,d__Bacteria;p__Bacillota_A;c__Clostridia;o__La...,1.9465,1.7881,98.05,0.052
89,d__Bacteria;p__Bacillota_A;c__Clostridia;o__Os...,1.8609,1.0262,99.15,0.050
90,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,1.6477,1.3536,97.12,0.044
91,d__Bacteria;p__Bacillota_A;c__Clostridia;o__La...,1.5230,1.2569,96.07,0.041
